In [6]:
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import pickle
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [7]:
data = pd.read_csv('../data/btc_data.csv', index_col=0)

In [8]:
def train_model(data, model, features, target, window_size):
      
      # Initialiser les listes pour stocker les prédictions et les vraies valeurs
      predictions = []
      actuals = []

      # Boucle à travers les données de la taille de la fenêtre jusqu'à la fin des données
      for i in range(window_size, len(data) - 1):
            # Diviser les données en ensembles d'entraînement et de test
            X_train = features.iloc[i-window_size:i, :]
            y_train = target.iloc[i-window_size:i]
            X_test = features.iloc[i:i+1, :]
            y_test = target.iloc[i]

            #Normaliser les données
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
            # Entraîner un modèle
            model.fit(X_train, y_train)
            
            # Faire une prédiction
            prediction = model.predict(X_test)[0]
            
            # Stocker les prédictions et les vraies valeurs
            predictions.append(prediction)
            actuals.append(y_test)

      # Évaluer le modèle
      accuracy = accuracy_score(actuals, predictions)
      print(f'Model Accuracy: {accuracy * 100:.2f}%')

In [21]:
def earn_metric(predicted_probs, progressions, n_days, i):
      base = c = 1
      for j in range(n_days):
            index = len(predicted_probs) - n_days + j
            c *= predicted_probs[index] * progressions[index] + (1 - predicted_probs[index])
            base *= progressions[index]
      return c / base


def train_model_proba_metric(data, model, features, target, window_size, n_days):

      # Initialiser les listes pour stocker les probaabilités prédites et les vraies valeurs
      predicted_probs = []
      progressions = []
      metric = []

      # Boucle à travers les données de la taille de la fenêtre jusqu'à la fin des données
      for i in range(window_size, len(data) - 1):
            # Diviser les données en ensembles d'entraînement et de test
            X_train = features.iloc[i-window_size:i, :]
            y_train = target.iloc[i-window_size:i]
            X_test = features.iloc[i:i+1, :]
            y_test = target.iloc[i]

            # Normaliser les données
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
            # Entraîner un modèle
            model.fit(X_train, y_train)

            # Obtenir les probabilités prédites pour la classe positive
            prediction_prob = model.predict_proba(X_test)[:, 1]
            
            # Stocker les probabilités prédites et les vraies valeurs
            predicted_probs.extend(prediction_prob)
            
            # Récupérer la progression réelle
            progressions.append(data.iloc[i]['progression tomorrow']+1)

            if i >= window_size + n_days:
                  metric.append(earn_metric(predicted_probs, progressions, n_days, i))

            # Sauvegardez les modèles dans un fichier
            filename = f'../models/xgboost_models/xgboost_{data.index[i]}.pkl'
            pickle.dump(model, open(filename, 'wb'))
      
      return np.mean(metric) 

In [22]:
# Sélectionner les caractéristiques et exclure la dernière ligne
features = data.drop(columns=['progression tomorrow', 'target', 'close', 'high', 'low', 'volumefrom', 'market_cap', 'difficulty']).iloc[:-1, :]
target = data['target'].iloc[:-1]
window_size = 1500
n_days = 1

In [23]:
best_params_xgb = {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}

xgboost = XGBClassifier(**best_params_xgb, random_state = 42)

In [25]:
train_model_proba_metric(data, xgboost, features, target, window_size, n_days)

1.002236645020043

In [ ]:
param_grid_xgb = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore', category=FutureWarning, module='sklearn')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
from IPython.display import display, HTML


# Créer une liste de combinaisons de paramètres
grid_list = list(ParameterGrid(param_grid_xgb))

# Pour boucler sur chaque combinaison :
best_accuracy = 0
best_params = None

for params in grid_list:
    try:
        # Instancier le modèle avec les paramètres
        model_instance = XGBClassifier(**params)
        
        # Appliquer la fonction train_model
        accuracy = train_model_grid(data, model_instance, features, target, window_size)
        
        # Si le modèle actuel a une meilleure précision que le précédent meilleur modèle, stocker sa précision et ses paramètres
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
            print(f"Nouveaux meilleurs paramètres trouvés : {params}, accuracy : {accuracy}")

        else:
            print(f'Trop nul la honte : {params}, accuracy : {accuracy}"')

        
            
    except Exception as e:
        # Gérer les combinaisons de paramètres non compatibles
        error_message = f"Error with parameters {params}: {e}"
        

print(f"Best Model Accuracy: {best_accuracy * 100:.5f}%")
print(f"Best Parameters: {best_params}")

Nouveaux meilleurs paramètres trouvés : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}, accuracy : 0.5887799564270153
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}, accuracy : 0.5833333333333334"
Nouveaux meilleurs paramètres trouvés : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}, accuracy : 0.5999455337690632
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}, accuracy : 0.5923202614379085"
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}, accuracy : 0.593681917211329"
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}, accuracy : 0.5874183006535948"
Nouveaux meilleurs paramètr

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore', category=FutureWarning, module='sklearn')
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
from IPython.display import display, HTML


# Créer une liste de combinaisons de paramètres
grid_list = list(ParameterGrid(param_grid_xgb))

# Pour boucler sur chaque combinaison :
best_earning = 0
best_params = None

for params in grid_list:
    try:
        # Instancier le modèle avec les paramètres
        model_instance = XGBClassifier(**params, random_state = 42)
        
        # Appliquer la fonction train_model
        earning = train_model_proba_metric(data, model_instance, features, target, window_size, n_days)
        
        # Si le modèle actuel a une meilleure précision que le précédent meilleur modèle, stocker sa précision et ses paramètres
        if earning > best_earning:
            best_earning = earning
            best_params = params
            print(f"Nouveaux meilleurs paramètres trouvés : {params}, earning : {earning}")

        else:
            print(f'Trop nul la honte : {params}, earning : {earning}')

        
            
    except Exception as e:
        # Gérer les combinaisons de paramètres non compatibles
        error_message = f"Error with parameters {params}: {e}"
        

print(f"Best Earnings: {best_earning * 100:.5f}%")
print(f"Best Parameters: {best_params}")

Nouveaux meilleurs paramètres trouvés : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}, earning : 1.0084009335278077
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}, earning : 1.0083829500965855"
Nouveaux meilleurs paramètres trouvés : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}, earning : 1.0205983479391452
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}, earning : 1.020363818651446"
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}, earning : 1.013274166001476"
Trop nul la honte : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}, earning : 1.0135417707667784"
Nouveaux meilleurs paramètres trou